In [5]:
%pip list

Package                                 Version            Editable project location
--------------------------------------- ------------------ ------------------------------------------------------------------------
absl-py                                 2.1.0
aext-assistant                          4.0.15
aext-assistant-server                   4.0.15
aext-core                               4.0.15
aext-core-server                        4.0.15
aext-panels                             4.0.15
aext-panels-server                      4.0.15
aext-share-notebook                     4.0.15
aext-share-notebook-server              4.0.15
aext-shared                             4.0.15
aiobotocore                             2.12.3
aiohappyeyeballs                        2.3.6
aiohttp                                 3.10.3
aioice                                  0.9.0
aioitertools                            0.7.1
aiortc                                  1.9.0
aiosignal                           

In [8]:
from pydub import AudioSegment
import webvtt

import os

# Define the file paths with direct paths
vtt_file = "c:/Users/shawn/OneDrive/Desktop/newbie/-uleG_Vecis.vtt"
audio_file = "c:/Users/shawn/OneDrive/Desktop/newbie/100+ Computer Science Concepts Explained.wav"

# Create output directory if it doesn't exist
output_dir = "c:/Users/shawn/OneDrive/Desktop/newbie/chunks"
os.makedirs(output_dir, exist_ok=True)

# Check if files exist
if not os.path.exists(vtt_file):
    print(f"Looking for VTT file at: {vtt_file}")
    raise FileNotFoundError(f"VTT file not found at: {vtt_file}")

# Load transcript with timestamps
vtt = webvtt.read(vtt_file)
if not os.path.exists(audio_file):
    raise FileNotFoundError(f"Audio file not found at: {audio_file}")

audio = AudioSegment.from_file(audio_file)

# Create transcript dictionary
transcript = {
    "segments": [{
        "words": [{
            "word": caption.text,
            "start": caption.start_in_seconds,
            "end": caption.end_in_seconds  # Add end time
        }]
    } for caption in vtt.captions]
}

# Audio is already loaded above, remove this duplicate line
chunks = []

current_chunk = []
current_start = 0

for segment in transcript["segments"]:
    for word in segment["words"]:
        if word["end"] - current_start > 15:  # Check if chunk exceeds 15 seconds
            # Save current chunk
            chunks.append({
                "text": " ".join([w["word"] for w in current_chunk]),
                "start": current_start,
                "end": word["end"]
            })
            # Start new chunk with overlap
            current_chunk = current_chunk[-2:]  # Keep last 2 words for overlap
            current_start = current_chunk[0]["start"]
        current_chunk.append(word)

# Save the last chunk
if current_chunk:
    chunks.append({
        "text": " ".join([w["word"] for w in current_chunk]),
        "start": current_start,
        "end": current_chunk[-1]["end"]
    })

# Export audio chunks
for i, chunk in enumerate(chunks):
    start_ms = chunk["start"] * 1000
    end_ms = chunk["end"] * 1000
    chunk_audio = audio[start_ms:end_ms]
    output_path = os.path.join(output_dir, f"chunk_{i}.mp3")
    chunk_audio.export(output_path, format="mp3")

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [2]:
import yt_dlp
import whisperx
import torch
from pydub import AudioSegment
import numpy as np
from transformers import pipeline
import webrtcvad
import wave
import contextlib
from scipy.io import wavfile
import json

class VideoSegmenter:
    def __init__(self, max_segment_length=15):
        self.max_segment_length = max_segment_length
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        # Initialize VAD
        self.vad = webrtcvad.Vad(3)  # Aggressiveness level 3
        # Initialize sentence transformer for semantic similarity
        self.sentence_analyzer = pipeline("feature-extraction", 
                                        model="sentence-transformers/all-MiniLM-L6-v2",
                                        device=self.device)
    
    def download_video(self, url):
        """Download video and extract audio using yt-dlp"""
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
            'outtmpl': 'audio.%(ext)s'
        }
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return 'audio.wav'

    def transcribe_audio(self, audio_path):
        """Transcribe audio using WhisperX for better alignment"""
        # Load WhisperX model
        model = whisperx.load_model("large-v2", device=self.device)
        
        # Transcribe with word-level timestamps
        result = model.transcribe(audio_path, batch_size=16)
        
        # Align whisper output
        model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=self.device)
        result = whisperx.align(result["segments"], model_a, metadata, audio_path, device=self.device)
        
        return result

    def detect_voice_activity(self, audio_path, window_size=30):
        """Detect voice activity in audio using WebRTC VAD"""
        with contextlib.closing(wave.open(audio_path, 'rb')) as wf:
            num_frames = wf.getnframes()
            frame_rate = wf.getframerate()
            audio_duration = num_frames / float(frame_rate)
            
        rate, audio = wavfile.read(audio_path)
        samples_per_window = int(window_size * rate / 1000)
        
        voice_activity = []
        for start in range(0, len(audio), samples_per_window):
            is_speech = self.vad.is_speech(audio[start:start + samples_per_window].tobytes(), 
                                         sample_rate=rate)
            voice_activity.append(is_speech)
            
        return voice_activity

    def calculate_semantic_similarity(self, text1, text2):
        """Calculate semantic similarity between two text segments"""
        embeddings1 = self.sentence_analyzer(text1)
        embeddings2 = self.sentence_analyzer(text2)
        
        similarity = np.dot(embeddings1[0], embeddings2[0]) / \
                    (np.linalg.norm(embeddings1[0]) * np.linalg.norm(embeddings2[0]))
        return similarity

    def create_segments(self, transcription, voice_activity, audio_path):
        """Create semantically meaningful segments"""
        segments = []
        current_segment = {
            'start': transcription['segments'][0]['start'],
            'text': '',
            'words': []
        }
        
        for segment in transcription['segments']:
            for word in segment['words']:
                # Check if adding this word would exceed max duration
                if word['end'] - current_segment['start'] > self.max_segment_length:
                    # Finalize current segment
                    current_segment['end'] = word['start']
                    segments.append(current_segment)
                    
                    # Start new segment
                    current_segment = {
                        'start': word['start'],
                        'text': '',
                        'words': []
                    }
                
                # Add word to current segment
                current_segment['words'].append(word)
                current_segment['text'] += ' ' + word['word']
        
        # Add final segment
        if current_segment['words']:
            current_segment['end'] = current_segment['words'][-1]['end']
            segments.append(current_segment)
        
        # Post-process segments based on semantic similarity and voice activity
        final_segments = self.optimize_segments(segments, voice_activity)
        
        return final_segments

    def optimize_segments(self, segments, voice_activity):
        """Optimize segments based on semantic coherence and voice activity"""
        optimized_segments = []
        current_segment = segments[0]
        
        for i in range(1, len(segments)):
            # Calculate semantic similarity with next segment
            similarity = self.calculate_semantic_similarity(
                current_segment['text'], 
                segments[i]['text']
            )
            
            # Check voice activity at segment boundary
            boundary_time = segments[i]['start']
            va_index = int(boundary_time * 1000 / 30)  # Assuming 30ms windows
            has_voice = voice_activity[va_index] if va_index < len(voice_activity) else False
            
            # Merge segments if they are semantically similar and have continuous voice activity
            if similarity > 0.8 and has_voice:
                current_segment['end'] = segments[i]['end']
                current_segment['text'] += ' ' + segments[i]['text']
                current_segment['words'].extend(segments[i]['words'])
            else:
                optimized_segments.append(current_segment)
                current_segment = segments[i]
        
        optimized_segments.append(current_segment)
        return optimized_segments

    def process_video(self, url, output_file='segments.json'):
        """Main processing pipeline"""
        # Download video and extract audio
        audio_path = self.download_video(url)
        
        # Transcribe audio
        transcription = self.transcribe_audio(audio_path)
        
        # Detect voice activity
        voice_activity = self.detect_voice_activity(audio_path)
        
        # Create segments
        segments = self.create_segments(transcription, voice_activity, audio_path)
        
        # Save results
        with open(output_file, 'w') as f:
            json.dump(segments, f, indent=2)
        
        return segments

In [3]:
segmenter = VideoSegmenter(max_segment_length=15)
segments = segmenter.process_video("https://www.youtube.com/watch?v=uleG_Vecis")


C:\Users\shawn\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[youtube:truncated_id] Extracting URL: https://www.youtube.com/watch?v=uleG_Vecis


ERROR: [youtube:truncated_id] uleG_Vecis: Incomplete YouTube ID uleG_Vecis. URL https://www.youtube.com/watch?v=uleG_Vecis looks truncated.


DownloadError: ERROR: [youtube:truncated_id] uleG_Vecis: Incomplete YouTube ID uleG_Vecis. URL https://www.youtube.com/watch?v=uleG_Vecis looks truncated.

In [4]:
import logging
from pydub import AudioSegment
import webrtcvad
import numpy as np
import wave
import contextlib
import datetime
import re
from typing import List, Dict, Tuple
import os

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('audio_segmentation.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class AudioSegmenter:
    def __init__(self, audio_path: str, vtt_path: str, output_dir: str = "segments"):
        """
        Initialize the audio segmenter
        
        Args:
            audio_path: Path to WAV audio file
            vtt_path: Path to VTT transcript file
            output_dir: Directory to save segmented audio files
        """
        self.audio_path = audio_path
        self.vtt_path = vtt_path
        self.output_dir = output_dir
        self.vad = webrtcvad.Vad(3)  # Aggressiveness level 3
        
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Created output directory: {output_dir}")

    def parse_vtt(self) -> List[Dict]:
        """Parse VTT file and extract timestamps and text"""
        logger.info(f"Parsing VTT file: {self.vtt_path}")
        segments = []
        
        try:
            with open(self.vtt_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # Split into segments
            blocks = content.split('\n\n')
            timestamp_pattern = r'(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3})'
            
            for block in blocks:
                if not block.strip() or block.strip() == 'WEBVTT':
                    continue
                    
                lines = block.strip().split('\n')
                timestamp_match = re.search(timestamp_pattern, lines[0])
                
                if timestamp_match:
                    start_time = self._time_to_seconds(timestamp_match.group(1))
                    end_time = self._time_to_seconds(timestamp_match.group(2))
                    text = ' '.join(lines[1:])
                    
                    segments.append({
                        'start': start_time,
                        'end': end_time,
                        'text': text
                    })
            
            logger.info(f"Successfully parsed {len(segments)} segments from VTT")
            return segments
            
        except Exception as e:
            logger.error(f"Error parsing VTT file: {str(e)}")
            raise

    def _time_to_seconds(self, time_str: str) -> float:
        """Convert timestamp string to seconds"""
        time_obj = datetime.datetime.strptime(time_str, '%H:%M:%S.%f')
        return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond/1000000

    def detect_voice_activity(self, audio_segment: AudioSegment, window_size: int = 30) -> List[bool]:
        """
        Detect voice activity in audio segment
        
        Args:
            audio_segment: Audio segment to analyze
            window_size: Size of window in milliseconds
            
        Returns:
            List of booleans indicating voice activity
        """
        logger.info("Detecting voice activity")
        try:
            # Convert audio to format compatible with WebRTC VAD
            audio_segment = audio_segment.set_channels(1)  # Convert to mono
            audio_segment = audio_segment.set_frame_rate(16000)  # Convert to 16kHz
            
            samples = np.array(audio_segment.get_array_of_samples())
            samples_per_window = int(window_size * 16000 / 1000)  # 16kHz sample rate
            
            voice_activity = []
            for start in range(0, len(samples), samples_per_window):
                chunk = samples[start:start + samples_per_window]
                if len(chunk) == samples_per_window:  # Only process full chunks
                    is_speech = self.vad.is_speech(chunk.tobytes(), sample_rate=16000)
                    voice_activity.append(is_speech)
            
            logger.info(f"Voice activity detection completed. Found {sum(voice_activity)} active windows")
            return voice_activity
            
        except Exception as e:
            logger.error(f"Error in voice activity detection: {str(e)}")
            raise

    def optimize_segments(self, segments: List[Dict], max_duration: float = 15.0) -> List[Dict]:
        """
        Optimize segments based on maximum duration and voice activity
        
        Args:
            segments: List of segments with start time, end time, and text
            max_duration: Maximum duration of each segment in seconds
            
        Returns:
            List of optimized segments
        """
        logger.info(f"Optimizing segments with max duration of {max_duration} seconds")
        optimized_segments = []
        current_segment = None
        
        try:
            for segment in segments:
                if current_segment is None:
                    current_segment = segment.copy()
                    continue
                
                merged_duration = segment['end'] - current_segment['start']
                
                if merged_duration > max_duration:
                    optimized_segments.append(current_segment)
                    current_segment = segment.copy()
                else:
                    current_segment['end'] = segment['end']
                    current_segment['text'] += ' ' + segment['text']
            
            if current_segment:
                optimized_segments.append(current_segment)
                
            logger.info(f"Segment optimization complete. Created {len(optimized_segments)} segments")
            return optimized_segments
            
        except Exception as e:
            logger.error(f"Error in segment optimization: {str(e)}")
            raise

    def save_segment(self, audio: AudioSegment, segment: Dict, index: int):
        """Save audio segment to file"""
        try:
            start_ms = int(segment['start'] * 1000)
            end_ms = int(segment['end'] * 1000)
            segment_audio = audio[start_ms:end_ms]
            
            # Create filename with timestamp information
            filename = f"segment_{index:03d}_{start_ms:07d}_{end_ms:07d}.wav"
            filepath = os.path.join(self.output_dir, filename)
            
            segment_audio.export(filepath, format="wav")
            logger.info(f"Saved segment {index}: {filepath}")
            
            # Save corresponding text
            text_filepath = os.path.join(self.output_dir, f"segment_{index:03d}.txt")
            with open(text_filepath, 'w', encoding='utf-8') as f:
                f.write(segment['text'])
            
            return filepath
            
        except Exception as e:
            logger.error(f"Error saving segment {index}: {str(e)}")
            raise

    def process(self, max_duration: float = 15.0) -> List[Tuple[str, str]]:
        """
        Process audio file and create segments
        
        Args:
            max_duration: Maximum duration of each segment in seconds
            
        Returns:
            List of tuples containing (audio_path, text_path) for each segment
        """
        logger.info(f"Starting audio segmentation process for {self.audio_path}")
        try:
            # Load audio file
            audio = AudioSegment.from_wav(self.audio_path)
            logger.info(f"Loaded audio file: duration={len(audio)/1000:.2f}s")
            
            # Parse VTT file
            segments = self.parse_vtt()
            
            # Optimize segments
            optimized_segments = self.optimize_segments(segments, max_duration)
            
            # Save segments
            output_files = []
            for i, segment in enumerate(optimized_segments):
                audio_path = self.save_segment(audio, segment, i)
                text_path = os.path.join(self.output_dir, f"segment_{i:03d}.txt")
                output_files.append((audio_path, text_path))
            
            logger.info(f"Processing complete. Created {len(output_files)} segments")
            return output_files
            
        except Exception as e:
            logger.error(f"Error in main processing: {str(e)}")
            raise

# Example usage
if __name__ == "__main__":
    segmenter = AudioSegmenter(
        audio_path="100+ Computer Science Concepts Explained.wav",
        vtt_path="-uleG_Vecis.vtt",
        output_dir="segments"
    )
    
    try:
        output_files = segmenter.process(max_duration=15.0)
        logger.info("Segmentation completed successfully")
        for audio_path, text_path in output_files:
            logger.info(f"Created segment pair: {audio_path} - {text_path}")
    except Exception as e:
        logger.error(f"Segmentation failed: {str(e)}")

INFO:__main__:Created output directory: segments
INFO:__main__:Starting audio segmentation process for 100+ Computer Science Concepts Explained.wav
INFO:__main__:Loaded audio file: duration=787.30s
INFO:__main__:Parsing VTT file: -uleG_Vecis.vtt
INFO:__main__:Successfully parsed 417 segments from VTT
INFO:__main__:Optimizing segments with max duration of 15.0 seconds
INFO:__main__:Segment optimization complete. Created 65 segments
INFO:__main__:Saved segment 0: segments\segment_000_0000080_0013359.wav
INFO:__main__:Saved segment 1: segments\segment_001_0011279_0024719.wav
INFO:__main__:Saved segment 2: segments\segment_002_0023519_0037679.wav
INFO:__main__:Saved segment 3: segments\segment_003_0035759_0050399.wav
INFO:__main__:Saved segment 4: segments\segment_004_0048479_0063280.wav
INFO:__main__:Saved segment 5: segments\segment_005_0060878_0074479.wav
INFO:__main__:Saved segment 6: segments\segment_006_0072799_0087200.wav
INFO:__main__:Saved segment 7: segments\segment_007_0085040_0

In [5]:
import lancedb
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import json
import logging
from typing import List, Dict, Union
import torch

class SegmentVectorStore:
    def __init__(self, segments_dir: str, db_path: str = "segments_db"):
        """
        Initialize the vector store for audio segments
        
        Args:
            segments_dir: Directory containing audio segments and transcripts
            db_path: Path to store LanceDB database
        """
        self.segments_dir = segments_dir
        self.db_path = db_path
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Initialize sentence transformer for embeddings
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)
        
        # Initialize LanceDB
        self.db = lancedb.connect(db_path)
        
        # Configure logging
        self.logger = logging.getLogger(__name__)
        
    def create_table(self, table_name: str = "segments") -> None:
        """Create or get LanceDB table"""
        try:
            # Define schema for the table
            schema = {
                "id": "string",
                "text": "string",
                "audio_path": "string",
                "start_time": "float",
                "end_time": "float",
                "vector": "vector(384)"  # dimension of all-MiniLM-L6-v2 embeddings
            }
            
            # Create table if it doesn't exist
            if table_name not in self.db.table_names():
                self.table = self.db.create_table(table_name, schema=schema)
                self.logger.info(f"Created new table: {table_name}")
            else:
                self.table = self.db.open_table(table_name)
                self.logger.info(f"Opened existing table: {table_name}")
                
        except Exception as e:
            self.logger.error(f"Error creating/opening table: {str(e)}")
            raise

    def load_segments(self) -> List[Dict]:
        """Load all segments from the segments directory"""
        segments = []
        try:
            # Get all segment files
            files = os.listdir(self.segments_dir)
            audio_files = [f for f in files if f.endswith('.wav')]
            
            for audio_file in audio_files:
                # Get corresponding text file
                base_name = audio_file.rsplit('.', 1)[0]
                text_file = f"{base_name}.txt"
                
                if text_file in files:
                    # Parse timestamp information from filename
                    # Format: segment_XXX_SSSSSSS_EEEEEEE.wav
                    parts = base_name.split('_')
                    start_time = float(parts[2]) / 1000  # Convert ms to seconds
                    end_time = float(parts[3]) / 1000
                    
                    # Read text content
                    with open(os.path.join(self.segments_dir, text_file), 'r', encoding='utf-8') as f:
                        text = f.read().strip()
                    
                    segments.append({
                        'id': base_name,
                        'text': text,
                        'audio_path': os.path.join(self.segments_dir, audio_file),
                        'start_time': start_time,
                        'end_time': end_time
                    })
            
            self.logger.info(f"Loaded {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error loading segments: {str(e)}")
            raise

    def create_embeddings(self, segments: List[Dict]) -> List[Dict]:
        """Create embeddings for segments"""
        try:
            # Get text from segments
            texts = [seg['text'] for seg in segments]
            
            # Create embeddings
            embeddings = self.model.encode(texts)
            
            # Add embeddings to segments
            for seg, emb in zip(segments, embeddings):
                seg['vector'] = emb
            
            self.logger.info(f"Created embeddings for {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error creating embeddings: {str(e)}")
            raise

    def index_segments(self, table_name: str = "segments") -> None:
        """Index all segments in the vector store"""
        try:
            # Create/open table
            self.create_table(table_name)
            
            # Load segments
            segments = self.load_segments()
            
            # Create embeddings
            segments_with_embeddings = self.create_embeddings(segments)
            
            # Add to LanceDB
            self.table.add(segments_with_embeddings)
            
            self.logger.info(f"Indexed {len(segments)} segments in LanceDB")
            
        except Exception as e:
            self.logger.error(f"Error indexing segments: {str(e)}")
            raise

    def search(self, query: str, limit: int = 5) -> List[Dict]:
        """
        Search for similar segments using a text query
        
        Args:
            query: Text query to search for
            limit: Maximum number of results to return
            
        Returns:
            List of most similar segments with their metadata
        """
        try:
            # Create query embedding
            query_embedding = self.model.encode(query)
            
            # Search in LanceDB
            results = self.table.search(query_embedding)\
                              .limit(limit)\
                              .select(["id", "text", "audio_path", "start_time", "end_time", "_distance"])\
                              .to_list()
            
            self.logger.info(f"Found {len(results)} results for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error searching segments: {str(e)}")
            raise

    def semantic_range_search(self, query: str, distance_threshold: float = 0.3) -> List[Dict]:
        """
        Search for segments within a semantic distance threshold
        
        Args:
            query: Text query to search for
            distance_threshold: Maximum semantic distance to include in results
            
        Returns:
            List of segments within the distance threshold
        """
        try:
            query_embedding = self.model.encode(query)
            
            results = self.table.search(query_embedding)\
                              .where(f"_distance < {distance_threshold}")\
                              .select(["id", "text", "audio_path", "start_time", "end_time", "_distance"])\
                              .to_list()
            
            self.logger.info(f"Found {len(results)} results within distance {distance_threshold} for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error in semantic range search: {str(e)}")
            raise

# Example usage
if __name__ == "__main__":
    # Initialize vector store
    vector_store = SegmentVectorStore(
        segments_dir="segments",
        db_path="segments_db"
    )
    
    try:
        # Index segments
        vector_store.index_segments()
        
        # Example search
        query = "turning machine"
        results = vector_store.search(query, limit=5)
        
        print("\nSearch Results:")
        for result in results:
            print(f"\nSegment ID: {result['id']}")
            print(f"Text: {result['text']}")
            print(f"Audio Path: {result['audio_path']}")
            print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")
            print(f"Distance: {result['_distance']:.4f}")
            
    except Exception as e:
        print(f"Error: {str(e)}")

INFO:datasets:PyTorch version 2.4.1+cpu available.
INFO:datasets:TensorFlow version 2.18.0 available.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
C:\Users\shawn\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
ERROR:__main__:Error creating/opening table: Schema must be an instance of pyarrow.Schema
ERROR:__main__:Error indexing segments: Schema must be an instance of pyarrow.Schema


Error: Schema must be an instance of pyarrow.Schema


In [6]:
import lancedb
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import json
import logging
from typing import List, Dict, Union
import torch
import pyarrow as pa

class SegmentVectorStore:
    def __init__(self, segments_dir: str, db_path: str = "segments_db"):
        """
        Initialize the vector store for audio segments
        
        Args:
            segments_dir: Directory containing audio segments and transcripts
            db_path: Path to store LanceDB database
        """
        self.segments_dir = segments_dir
        self.db_path = db_path
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Initialize sentence transformer for embeddings
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)
        
        # Initialize LanceDB
        self.db = lancedb.connect(db_path)
        
        # Configure logging
        self.logger = logging.getLogger(__name__)
        
    def create_table(self, table_name: str = "segments") -> None:
        """Create or get LanceDB table"""
        try:
            # Define schema using PyArrow
            schema = pa.schema([
                pa.field('id', pa.string()),
                pa.field('text', pa.string()),
                pa.field('audio_path', pa.string()),
                pa.field('start_time', pa.float32()),
                pa.field('end_time', pa.float32()),
                pa.field('vector', pa.list_(pa.float32(), 384))  # dimension of all-MiniLM-L6-v2 embeddings
            ])
            
            # Create table if it doesn't exist
            if table_name not in self.db.table_names():
                self.table = self.db.create_table(table_name, schema=schema)
                self.logger.info(f"Created new table: {table_name}")
            else:
                self.table = self.db.open_table(table_name)
                self.logger.info(f"Opened existing table: {table_name}")
                
        except Exception as e:
            self.logger.error(f"Error creating/opening table: {str(e)}")
            raise

    def load_segments(self) -> List[Dict]:
        """Load all segments from the segments directory"""
        segments = []
        try:
            # Get all segment files
            files = os.listdir(self.segments_dir)
            audio_files = [f for f in files if f.endswith('.wav')]
            
            for audio_file in audio_files:
                # Get corresponding text file
                base_name = audio_file.rsplit('.', 1)[0]
                text_file = f"{base_name}.txt"
                
                if text_file in files:
                    # Parse timestamp information from filename
                    # Format: segment_XXX_SSSSSSS_EEEEEEE.wav
                    parts = base_name.split('_')
                    start_time = float(parts[2]) / 1000  # Convert ms to seconds
                    end_time = float(parts[3]) / 1000
                    
                    # Read text content
                    with open(os.path.join(self.segments_dir, text_file), 'r', encoding='utf-8') as f:
                        text = f.read().strip()
                    
                    segments.append({
                        'id': base_name,
                        'text': text,
                        'audio_path': os.path.join(self.segments_dir, audio_file),
                        'start_time': start_time,
                        'end_time': end_time
                    })
            
            self.logger.info(f"Loaded {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error loading segments: {str(e)}")
            raise

    def create_embeddings(self, segments: List[Dict]) -> List[Dict]:
        """Create embeddings for segments"""
        try:
            # Get text from segments
            texts = [seg['text'] for seg in segments]
            
            # Create embeddings
            embeddings = self.model.encode(texts)
            
            # Convert embeddings to list format for PyArrow compatibility
            for seg, emb in zip(segments, embeddings):
                seg['vector'] = emb.tolist()  # Convert numpy array to list
            
            self.logger.info(f"Created embeddings for {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error creating embeddings: {str(e)}")
            raise

    def index_segments(self, table_name: str = "segments") -> None:
        """Index all segments in the vector store"""
        try:
            # Create/open table
            self.create_table(table_name)
            
            # Load segments
            segments = self.load_segments()
            
            # Create embeddings
            segments_with_embeddings = self.create_embeddings(segments)
            
            # Convert to PyArrow table and add to LanceDB
            data = pa.Table.from_pylist(segments_with_embeddings)
            self.table.add(data)
            
            self.logger.info(f"Indexed {len(segments)} segments in LanceDB")
            
        except Exception as e:
            self.logger.error(f"Error indexing segments: {str(e)}")
            raise

    def search(self, query: str, limit: int = 5) -> List[Dict]:
        """
        Search for similar segments using a text query
        
        Args:
            query: Text query to search for
            limit: Maximum number of results to return
            
        Returns:
            List of most similar segments with their metadata
        """
        try:
            # Create query embedding
            query_embedding = self.model.encode(query).tolist()  # Convert to list for compatibility
            
            # Search in LanceDB
            results = self.table.search(query_embedding)\
                              .limit(limit)\
                              .select(["id", "text", "audio_path", "start_time", "end_time", "_distance"])\
                              .to_list()
            
            self.logger.info(f"Found {len(results)} results for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error searching segments: {str(e)}")
            raise

    def semantic_range_search(self, query: str, distance_threshold: float = 0.3) -> List[Dict]:
        """
        Search for segments within a semantic distance threshold
        
        Args:
            query: Text query to search for
            distance_threshold: Maximum semantic distance to include in results
            
        Returns:
            List of segments within the distance threshold
        """
        try:
            query_embedding = self.model.encode(query).tolist()  # Convert to list for compatibility
            
            results = self.table.search(query_embedding)\
                              .where(f"_distance < {distance_threshold}")\
                              .select(["id", "text", "audio_path", "start_time", "end_time", "_distance"])\
                              .to_list()
            
            self.logger.info(f"Found {len(results)} results within distance {distance_threshold} for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error in semantic range search: {str(e)}")
            raise

# Example usage
if __name__ == "__main__":
    # Initialize vector store
    vector_store = SegmentVectorStore(
        segments_dir="segments",
        db_path="segments_db"
    )
    
    try:
        # Index segments
        vector_store.index_segments()
        
        # Example search
        query = "turing machine"
        results = vector_store.search(query, limit=5)
        
        print("\nSearch Results:")
        for result in results:
            print(f"\nSegment ID: {result['id']}")
            print(f"Text: {result['text']}")
            print(f"Audio Path: {result['audio_path']}")
            print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")
            print(f"Distance: {result['_distance']:.4f}")
            
    except Exception as e:
        print(f"Error: {str(e)}")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Created new table: segments
INFO:__main__:Loaded 0 segments


Batches: 0it [00:00, ?it/s]

INFO:__main__:Created embeddings for 0 segments
ERROR:__main__:Error indexing segments: 'Field "id" does not exist in schema'


Error: 'Field "id" does not exist in schema'


In [7]:
import lancedb
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import json
import logging
from typing import List, Dict, Union
import torch
import pyarrow as pa

class SegmentVectorStore:
    def __init__(self, segments_dir: str, db_path: str = "segments_db"):
        """
        Initialize the vector store for audio segments
        """
        self.segments_dir = segments_dir
        self.db_path = db_path
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Initialize sentence transformer for embeddings
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)
        
        # Initialize LanceDB
        self.db = lancedb.connect(db_path)
        
        # Configure logging
        self.logger = logging.getLogger(__name__)

    def load_segments(self) -> List[Dict]:
        """Load all segments from the segments directory"""
        segments = []
        try:
            # Check if directory exists
            if not os.path.exists(self.segments_dir):
                self.logger.error(f"Directory not found: {self.segments_dir}")
                return segments

            # Get all segment files
            files = os.listdir(self.segments_dir)
            audio_files = [f for f in files if f.endswith('.wav')]
            
            if not audio_files:
                self.logger.warning(f"No WAV files found in {self.segments_dir}")
                return segments

            for audio_file in audio_files:
                # Get corresponding text file
                base_name = audio_file.rsplit('.', 1)[0]
                text_file = f"{base_name}.txt"
                
                if text_file in files:
                    try:
                        # Parse timestamp information from filename
                        # Format: segment_XXX_SSSSSSS_EEEEEEE.wav
                        parts = base_name.split('_')
                        if len(parts) >= 4:  # Ensure filename has enough parts
                            start_time = float(parts[2]) / 1000  # Convert ms to seconds
                            end_time = float(parts[3]) / 1000
                            
                            # Read text content
                            text_path = os.path.join(self.segments_dir, text_file)
                            with open(text_path, 'r', encoding='utf-8') as f:
                                text = f.read().strip()
                            
                            segments.append({
                                'segment_id': base_name,  # Changed from 'id' to 'segment_id'
                                'segment_text': text,     # Changed from 'text' to 'segment_text'
                                'audio_path': os.path.join(self.segments_dir, audio_file),
                                'start_time': start_time,
                                'end_time': end_time
                            })
                    except (IndexError, ValueError) as e:
                        self.logger.warning(f"Skipping malformed file {audio_file}: {str(e)}")
                        continue
            
            self.logger.info(f"Loaded {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error loading segments: {str(e)}")
            raise

    def create_embeddings(self, segments: List[Dict]) -> List[Dict]:
        """Create embeddings for segments"""
        try:
            if not segments:
                self.logger.warning("No segments to create embeddings for")
                return segments

            # Get text from segments
            texts = [seg['segment_text'] for seg in segments]
            
            # Create embeddings
            embeddings = self.model.encode(texts)
            
            # Convert embeddings to list format for PyArrow compatibility
            for seg, emb in zip(segments, embeddings):
                seg['vector'] = emb.tolist()  # Convert numpy array to list
            
            self.logger.info(f"Created embeddings for {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error creating embeddings: {str(e)}")
            raise

    def index_segments(self, table_name: str = "segments") -> None:
        """Index all segments in the vector store"""
        try:
            # Load segments first
            segments = self.load_segments()
            
            if not segments:
                self.logger.error("No segments found to index")
                return

            # Create embeddings
            segments_with_embeddings = self.create_embeddings(segments)
            
            # Define schema using segment structure
            schema = pa.schema([
                pa.field('segment_id', pa.string()),
                pa.field('segment_text', pa.string()),
                pa.field('audio_path', pa.string()),
                pa.field('start_time', pa.float32()),
                pa.field('end_time', pa.float32()),
                pa.field('vector', pa.list_(pa.float32(), 384))
            ])
            
            # Create table if it doesn't exist
            if table_name not in self.db.table_names():
                self.table = self.db.create_table(table_name, schema=schema)
                self.logger.info(f"Created new table: {table_name}")
            else:
                self.table = self.db.open_table(table_name)
                self.logger.info(f"Opened existing table: {table_name}")

            # Convert to PyArrow table and add to LanceDB
            data = pa.Table.from_pylist(segments_with_embeddings, schema=schema)
            self.table.add(data)
            
            self.logger.info(f"Indexed {len(segments)} segments in LanceDB")
            
        except Exception as e:
            self.logger.error(f"Error indexing segments: {str(e)}")
            raise

    def search(self, query: str, limit: int = 5) -> List[Dict]:
        """Search for similar segments using a text query"""
        try:
            # Create query embedding
            query_embedding = self.model.encode(query).tolist()
            
            # Search in LanceDB
            results = self.table.search(query_embedding)\
                              .limit(limit)\
                              .select(["segment_id", "segment_text", "audio_path", 
                                     "start_time", "end_time", "_distance"])\
                              .to_list()
            
            self.logger.info(f"Found {len(results)} results for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error searching segments: {str(e)}")
            raise

# Example usage
if __name__ == "__main__":
    import os
    
    # Get the current directory
    current_dir = os.getcwd()
    segments_dir = os.path.join(current_dir, "segments")
    
    # Print directory contents for debugging
    print(f"Looking for segments in: {segments_dir}")
    if os.path.exists(segments_dir):
        print("Contents of segments directory:")
        for file in os.listdir(segments_dir):
            print(f"  - {file}")
    else:
        print("Segments directory not found!")
    
    # Initialize vector store
    vector_store = SegmentVectorStore(
        segments_dir=segments_dir,
        db_path="segments_db"
    )
    
    try:
        # Index segments
        vector_store.index_segments()
        
        # Example search (only if segments were indexed)
        if os.path.exists(segments_dir) and os.listdir(segments_dir):
            query = "example query text"
            results = vector_store.search(query, limit=5)
            
            print("\nSearch Results:")
            for result in results:
                print(f"\nSegment ID: {result['segment_id']}")
                print(f"Text: {result['segment_text']}")
                print(f"Audio Path: {result['audio_path']}")
                print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")
                print(f"Distance: {result['_distance']:.4f}")
            
    except Exception as e:
        print(f"Error: {str(e)}")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Looking for segments in: c:\Users\shawn\OneDrive\Desktop\newbie\segments
Contents of segments directory:
  - segment_000.txt
  - segment_000_0000080_0013359.wav
  - segment_001.txt
  - segment_001_0011279_0024719.wav
  - segment_002.txt
  - segment_002_0023519_0037679.wav
  - segment_003.txt
  - segment_003_0035759_0050399.wav
  - segment_004.txt
  - segment_004_0048479_0063280.wav
  - segment_005.txt
  - segment_005_0060878_0074479.wav
  - segment_006.txt
  - segment_006_0072799_0087200.wav
  - segment_007.txt
  - segment_007_0085040_0099280.wav
  - segment_008.txt
  - segment_008_0097759_0110560.wav
  - segment_009.txt
  - segment_009_0108640_0123359.wav
  - segment_010.txt
  - segment_010_0121680_0135199.wav
  - segment_011.txt
  - segment_011_0133520_0148400.wav
  - segment_012.txt
  - segment_012_0146318_0159839.wav
  - segment_013.txt
  - segment_013_0157919_0172079.wav
  - segment_014.txt
  - segment_014_0170080_0184560.wav
  - segment_015.txt
  - segment_015_0182239_0196560.wav

INFO:__main__:Loaded 0 segments
ERROR:__main__:No segments found to index


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Error searching segments: 'SegmentVectorStore' object has no attribute 'table'


Error: 'SegmentVectorStore' object has no attribute 'table'


In [8]:
import lancedb
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import logging
from typing import List, Dict
import torch
import pyarrow as pa
import re

class SegmentVectorStore:
    def __init__(self, segments_dir: str, db_path: str = "segments_db"):
        self.segments_dir = segments_dir
        self.db_path = db_path
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)
        self.db = lancedb.connect(db_path)
        self.logger = logging.getLogger(__name__)
        self.table = None  # Initialize table attribute

    def load_segments(self) -> List[Dict]:
        segments = []
        try:
            files = os.listdir(self.segments_dir)
            # Modified pattern to match your file format
            pattern = r'segment_(\d+)_(\d+)_(\d+)\.wav'
            
            # Get all wav files
            wav_files = [f for f in files if f.endswith('.wav')]
            
            for wav_file in wav_files:
                match = re.match(pattern, wav_file)
                if match:
                    segment_num = match.group(1)
                    start_time = float(match.group(2)) / 1000
                    end_time = float(match.group(3)) / 1000
                    
                    # Get corresponding text file
                    txt_file = f"segment_{segment_num}.txt"
                    
                    if txt_file in files:
                        txt_path = os.path.join(self.segments_dir, txt_file)
                        with open(txt_path, 'r', encoding='utf-8') as f:
                            text = f.read().strip()
                        
                        segments.append({
                            'segment_id': f"segment_{segment_num}",
                            'segment_text': text,
                            'audio_path': os.path.join(self.segments_dir, wav_file),
                            'start_time': start_time,
                            'end_time': end_time
                        })
            
            self.logger.info(f"Loaded {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error loading segments: {str(e)}")
            raise

    def create_embeddings(self, segments: List[Dict]) -> List[Dict]:
        try:
            if not segments:
                self.logger.warning("No segments to create embeddings for")
                return segments

            texts = [seg['segment_text'] for seg in segments]
            embeddings = self.model.encode(texts)
            
            for seg, emb in zip(segments, embeddings):
                seg['vector'] = emb.tolist()
            
            self.logger.info(f"Created embeddings for {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error creating embeddings: {str(e)}")
            raise

    def index_segments(self, table_name: str = "segments") -> None:
        try:
            segments = self.load_segments()
            
            if not segments:
                self.logger.error("No segments found to index")
                return

            # Create embeddings
            segments_with_embeddings = self.create_embeddings(segments)
            
            # Define schema
            schema = pa.schema([
                pa.field('segment_id', pa.string()),
                pa.field('segment_text', pa.string()),
                pa.field('audio_path', pa.string()),
                pa.field('start_time', pa.float32()),
                pa.field('end_time', pa.float32()),
                pa.field('vector', pa.list_(pa.float32(), 384))
            ])
            
            # Create or open table
            if table_name not in self.db.table_names():
                self.table = self.db.create_table(table_name, schema=schema)
                self.logger.info(f"Created new table: {table_name}")
            else:
                self.table = self.db.open_table(table_name)
                self.logger.info(f"Opened existing table: {table_name}")

            # Convert to PyArrow table and add to LanceDB
            data = pa.Table.from_pylist(segments_with_embeddings, schema=schema)
            self.table.add(data)
            
            self.logger.info(f"Successfully indexed {len(segments)} segments in LanceDB")
            
        except Exception as e:
            self.logger.error(f"Error indexing segments: {str(e)}")
            raise

    def search(self, query: str, limit: int = 5) -> List[Dict]:
        try:
            if self.table is None:
                raise ValueError("Table not initialized. Please run index_segments first.")
                
            query_embedding = self.model.encode(query).tolist()
            
            results = self.table.search(query_embedding)\
                              .limit(limit)\
                              .select(["segment_id", "segment_text", "audio_path", 
                                     "start_time", "end_time", "_distance"])\
                              .to_list()
            
            self.logger.info(f"Found {len(results)} results for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error searching segments: {str(e)}")
            raise

# Example usage with better error handling
if __name__ == "__main__":
    import os
    
    current_dir = os.getcwd()
    segments_dir = os.path.join(current_dir, "segments")
    
    print(f"Looking for segments in: {segments_dir}")
    if os.path.exists(segments_dir):
        print("Contents of segments directory:")
        files = os.listdir(segments_dir)
        wav_files = [f for f in files if f.endswith('.wav')]
        txt_files = [f for f in files if f.endswith('.txt')]
        print(f"Found {len(wav_files)} WAV files and {len(txt_files)} TXT files")
    
    try:
        # Initialize vector store
        vector_store = SegmentVectorStore(
            segments_dir=segments_dir,
            db_path="segments_db"
        )
        
        # Index segments
        vector_store.index_segments()
        
        # Only proceed with search if indexing was successful
        if vector_store.table is not None:
            query = "what is this video about?"
            results = vector_store.search(query, limit=5)
            
            print("\nSearch Results:")
            for result in results:
                print(f"\nSegment ID: {result['segment_id']}")
                print(f"Text: {result['segment_text']}")
                print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")
                print(f"Similarity Score: {1 - result['_distance']:.4f}")
        
    except Exception as e:
        print(f"Error: {str(e)}")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Looking for segments in: c:\Users\shawn\OneDrive\Desktop\newbie\segments
Contents of segments directory:
Found 65 WAV files and 65 TXT files


INFO:__main__:Loaded 65 segments


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:Created embeddings for 65 segments
INFO:__main__:Opened existing table: segments
ERROR:__main__:Error indexing segments: 'Field "id" does not exist in schema'


Error: 'Field "id" does not exist in schema'


In [9]:
# Test the vector store with proper error handling
try:
	# Initialize vector store with existing segments directory
	vector_store = SegmentVectorStore(
		segments_dir=segments_dir,
		db_path="segments_db"
	)
	
	# Index the segments
	print("Indexing segments...")
	vector_store.index_segments()
	
	# Test search functionality with different queries
	test_queries = [
		"What is computer science?",
		"What is a Turing machine?",
		"How does object oriented programming work?",
		"Explain algorithms"
	]
	
	print("\nTesting search functionality with multiple queries:")
	for query in test_queries:
		print(f"\nQuery: {query}")
		results = vector_store.search(query, limit=3)
		
		for i, result in enumerate(results, 1):
			print(f"\nResult {i}:")
			print(f"Segment: {result['segment_id']}")
			print(f"Text: {result['segment_text'][:200]}...")  # Show first 200 chars
			print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")
			print(f"Similarity Score: {1 - result['_distance']:.4f}")

except Exception as e:
	print(f"Error during testing: {str(e)}")


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Loaded 65 segments


Indexing segments...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:Created embeddings for 65 segments
INFO:__main__:Opened existing table: segments
ERROR:__main__:Error indexing segments: 'Field "id" does not exist in schema'


Error during testing: 'Field "id" does not exist in schema'


In [10]:
import lancedb
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import logging
from typing import List, Dict
import torch
import pyarrow as pa
import re

class SegmentVectorStore:
    def __init__(self, segments_dir: str, db_path: str = "segments_db"):
        self.segments_dir = segments_dir
        self.db_path = db_path
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)
        self.db = lancedb.connect(db_path)
        self.logger = logging.getLogger(__name__)
        self.table = None

    def load_segments(self) -> List[Dict]:
        segments = []
        try:
            files = os.listdir(self.segments_dir)
            wav_files = [f for f in files if f.endswith('.wav')]
            
            for wav_file in wav_files:
                # Extract segment number from filename
                segment_num = wav_file.split('_')[1]  # Get the number after "segment_"
                txt_file = f"{wav_file.rsplit('_', 2)[0]}.txt"  # Remove timestamp parts
                
                if txt_file in files:
                    txt_path = os.path.join(self.segments_dir, txt_file)
                    with open(txt_path, 'r', encoding='utf-8') as f:
                        text = f.read().strip()
                    
                    # Extract timestamps from wav filename
                    parts = wav_file.split('_')
                    if len(parts) >= 4:
                        start_time = float(parts[-2]) / 1000
                        end_time = float(parts[-1].split('.')[0]) / 1000
                    else:
                        start_time = 0.0
                        end_time = 0.0
                    
                    segments.append({
                        'id': f"segment_{segment_num}",  # Changed from segment_id to id
                        'segment_text': text,
                        'audio_path': os.path.join(self.segments_dir, wav_file),
                        'start_time': start_time,
                        'end_time': end_time
                    })
            
            self.logger.info(f"Loaded {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error loading segments: {str(e)}")
            raise

    def create_embeddings(self, segments: List[Dict]) -> List[Dict]:
        try:
            if not segments:
                self.logger.warning("No segments to create embeddings for")
                return segments

            texts = [seg['segment_text'] for seg in segments]
            embeddings = self.model.encode(texts)
            
            for seg, emb in zip(segments, embeddings):
                seg['vector'] = emb.tolist()
            
            self.logger.info(f"Created embeddings for {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error creating embeddings: {str(e)}")
            raise

    def index_segments(self, table_name: str = "segments") -> None:
        try:
            segments = self.load_segments()
            
            if not segments:
                self.logger.error("No segments found to index")
                return

            segments_with_embeddings = self.create_embeddings(segments)
            
            # Updated schema with 'id' instead of 'segment_id'
            schema = pa.schema([
                pa.field('id', pa.string()),  # Changed from segment_id to id
                pa.field('segment_text', pa.string()),
                pa.field('audio_path', pa.string()),
                pa.field('start_time', pa.float32()),
                pa.field('end_time', pa.float32()),
                pa.field('vector', pa.list_(pa.float32(), 384))
            ])
            
            # Drop existing table if it exists
            if table_name in self.db.table_names():
                self.db.drop_table(table_name)
            
            # Create new table
            self.table = self.db.create_table(
                table_name,
                data=segments_with_embeddings,
                schema=schema,
                mode="overwrite"
            )
            
            self.logger.info(f"Successfully indexed {len(segments)} segments in LanceDB")
            
        except Exception as e:
            self.logger.error(f"Error indexing segments: {str(e)}")
            raise

    def search(self, query: str, limit: int = 5) -> List[Dict]:
        try:
            if self.table is None:
                raise ValueError("Table not initialized. Please run index_segments first.")
                
            query_embedding = self.model.encode(query).tolist()
            
            results = self.table.search(query_embedding)\
                              .limit(limit)\
                              .select(["id", "segment_text", "audio_path", 
                                     "start_time", "end_time", "_distance"])\
                              .to_list()
            
            self.logger.info(f"Found {len(results)} results for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error searching segments: {str(e)}")
            raise

In [11]:
vector_store = SegmentVectorStore(segments_dir="segments", db_path="segments_db")
vector_store.index_segments()
results = vector_store.search("turing machine")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Loaded 65 segments


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:Created embeddings for 65 segments
INFO:__main__:Successfully indexed 65 segments in LanceDB


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Error searching segments: LanceError(Schema): Column _distance does not exist, D:\a\lance\lance\rust\lance-core\src\datatypes\schema.rs:190:31


ValueError: LanceError(Schema): Column _distance does not exist, D:\a\lance\lance\rust\lance-core\src\datatypes\schema.rs:190:31

In [12]:
import lancedb
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import logging
from typing import List, Dict
import torch
import pyarrow as pa
import re

class SegmentVectorStore:
    def __init__(self, segments_dir: str, db_path: str = "segments_db"):
        self.segments_dir = segments_dir
        self.db_path = db_path
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)
        self.db = lancedb.connect(db_path)
        self.logger = logging.getLogger(__name__)
        self.table = None

    def load_segments(self) -> List[Dict]:
        segments = []
        try:
            files = os.listdir(self.segments_dir)
            wav_files = [f for f in files if f.endswith('.wav')]
            
            for wav_file in wav_files:
                # Extract segment number from filename
                segment_num = wav_file.split('_')[1]  # Get the number after "segment_"
                txt_file = f"{wav_file.rsplit('_', 2)[0]}.txt"  # Remove timestamp parts
                
                if txt_file in files:
                    txt_path = os.path.join(self.segments_dir, txt_file)
                    with open(txt_path, 'r', encoding='utf-8') as f:
                        text = f.read().strip()
                    
                    # Extract timestamps from wav filename
                    parts = wav_file.split('_')
                    if len(parts) >= 4:
                        start_time = float(parts[-2]) / 1000
                        end_time = float(parts[-1].split('.')[0]) / 1000
                    else:
                        start_time = 0.0
                        end_time = 0.0
                    
                    segments.append({
                        'id': f"segment_{segment_num}",
                        'segment_text': text,
                        'audio_path': os.path.join(self.segments_dir, wav_file),
                        'start_time': start_time,
                        'end_time': end_time
                    })
            
            self.logger.info(f"Loaded {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error loading segments: {str(e)}")
            raise

    def create_embeddings(self, segments: List[Dict]) -> List[Dict]:
        try:
            if not segments:
                self.logger.warning("No segments to create embeddings for")
                return segments

            texts = [seg['segment_text'] for seg in segments]
            embeddings = self.model.encode(texts)
            
            for seg, emb in zip(segments, embeddings):
                seg['vector'] = emb.tolist()
            
            self.logger.info(f"Created embeddings for {len(segments)} segments")
            return segments
            
        except Exception as e:
            self.logger.error(f"Error creating embeddings: {str(e)}")
            raise

    def index_segments(self, table_name: str = "segments") -> None:
        try:
            segments = self.load_segments()
            
            if not segments:
                self.logger.error("No segments found to index")
                return

            segments_with_embeddings = self.create_embeddings(segments)
            
            schema = pa.schema([
                pa.field('id', pa.string()),
                pa.field('segment_text', pa.string()),
                pa.field('audio_path', pa.string()),
                pa.field('start_time', pa.float32()),
                pa.field('end_time', pa.float32()),
                pa.field('vector', pa.list_(pa.float32(), 384))
            ])
            
            # Drop existing table if it exists
            if table_name in self.db.table_names():
                self.db.drop_table(table_name)
            
            # Create new table with vector column configuration
            self.table = self.db.create_table(
                table_name,
                data=segments_with_embeddings,
                schema=schema,
                mode="overwrite"
            )
            
            # Create vector index
            self.table.create_index(["vector"])
            
            self.logger.info(f"Successfully indexed {len(segments)} segments in LanceDB")
            
        except Exception as e:
            self.logger.error(f"Error indexing segments: {str(e)}")
            raise

    def search(self, query: str, limit: int = 5) -> List[Dict]:
        try:
            if self.table is None:
                raise ValueError("Table not initialized. Please run index_segments first.")
                
            query_embedding = self.model.encode(query).tolist()
            
            # Use vector_column parameter and don't select _distance
            results = self.table.search(
                query_embedding,
                vector_column_name="vector",
                limit=limit
            ).select([
                "id",
                "segment_text",
                "audio_path",
                "start_time",
                "end_time"
            ]).to_list()
            
            self.logger.info(f"Found {len(results)} results for query: {query}")
            return results
            
        except Exception as e:
            self.logger.error(f"Error searching segments: {str(e)}")
            raise

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    
    try:
        vector_store = SegmentVectorStore(segments_dir="segments", db_path="segments_db")
        vector_store.index_segments()
        
        # Test search
        if vector_store.table is not None:
            results = vector_store.search("what is this video about?", limit=5)
            print("\nSearch Results:")
            for result in results:
                print(f"\nSegment ID: {result['id']}")
                print(f"Text: {result['segment_text']}")
                print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")
                
    except Exception as e:
        print(f"Error: {str(e)}")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Loaded 65 segments


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:Created embeddings for 65 segments
ERROR:__main__:Error indexing segments: Metric ['vector'] not supported.


Error: Metric ['vector'] not supported.


In [13]:
# Initialize vector store and create table with proper vector index
vector_store = SegmentVectorStore(segments_dir=segments_dir, db_path="segments_db")

try:
    # Drop existing table if it exists
    if "segments" in vector_store.db.table_names():
        vector_store.db.drop_table("segments")
    
    # Load and process segments
    segments = vector_store.load_segments()
    segments_with_embeddings = vector_store.create_embeddings(segments)
    
    # Create table with vector column configuration
    vector_store.table = vector_store.db.create_table(
        name="segments",
        data=segments_with_embeddings,
        mode="overwrite",
        index_config={
            "vector": {
                "type": "FLAT",
                "metric_type": "L2"
            }
        }
    )
    
    # Test search functionality
    results = vector_store.search("what is computer science", limit=3)
    
    print("\nSearch Results:")
    for i, result in enumerate(results, 1):
        print(f"\nResult {i}:")
        print(f"Segment: {result['id']}")
        print(f"Text: {result['segment_text'][:200]}...")  # Show first 200 chars
        print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")

except Exception as e:
    print(f"Error: {str(e)}")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Loaded 65 segments


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:Created embeddings for 65 segments


Error: LanceDBConnection.create_table() got an unexpected keyword argument 'index_config'


In [14]:
def query_segments(query: str, limit: int = 3):
	"""
	Query the vector store and format results in a readable way
	"""
	try:
		# Ensure vector store is initialized
		if not hasattr(vector_store, 'table') or vector_store.table is None:
			print("Initializing vector store...")
			vector_store.index_segments()
		
		# Perform search
		results = vector_store.search(query, limit=limit)
		
		if not results:
			print("No relevant segments found.")
			return
			
		print(f"\nSearch Results for: '{query}'\n")
		print("-" * 80)
		
		for i, result in enumerate(results, 1):
			print(f"\nResult {i}:")
			print(f"Segment: {result['id']}")
			print(f"Time Range: {result['start_time']:.2f}s - {result['end_time']:.2f}s")
			print(f"Text: {result['segment_text']}")
			print("-" * 80)
			
	except Exception as e:
		print(f"Error during search: {str(e)}")

# Test the function with some example queries
test_queries = [
	"What is computer science?",
	"How does memory work in computers?",
	"Explain data structures",
	"What is object oriented programming?"
]

for query in test_queries:
	query_segments(query)
	print("\n" + "="*80 + "\n")


INFO:__main__:Loaded 65 segments


Initializing vector store...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:Created embeddings for 65 segments
ERROR:__main__:Error indexing segments: Metric ['vector'] not supported.


Error during search: Metric ['vector'] not supported.




...

ERROR:__main__:Error searching segments: LanceTable.search() got an unexpected keyword argument 'limit'


Error during search: LanceTable.search() got an unexpected keyword argument 'limit'




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Error searching segments: LanceTable.search() got an unexpected keyword argument 'limit'


Error during search: LanceTable.search() got an unexpected keyword argument 'limit'




In [15]:
def create_semantic_search_ui():
	# Helper function to format search results
	def format_result(result):
		"""Format a single search result for display"""
		return f"""
Time: {result['start_time']:.2f}s - {result['end_time']:.2f}s
Text: {result['segment_text']}
Audio: {os.path.basename(result['audio_path'])}
{'='*80}
"""

	# Function to handle search
	def search_segments(query, limit=5, threshold=0.6):
		try:
			# Ensure vector store is available
			if not hasattr(vector_store, 'table') or vector_store.table is None:
				print("Initializing vector store...")
				vector_store.index_segments()

			# Perform search 
			results = vector_store.search(query, limit=limit)
			
			if not results:
				return "No results found."
				
			# Format output
			output = f"\nSearch Results for: '{query}'\n"
			output += "="*80 + "\n"
			for i, result in enumerate(results, 1):
				output += f"\nResult {i}:\n"
				output += format_result(result)
			
			return output

		except Exception as e:
			return f"Error during search: {str(e)}"

	# Test queries
	print("Testing semantic search with example queries...\n")
	
	test_queries = [
		"What is object oriented programming?",
		"How does memory work in computers?",
		"Explain data structures",
		"What are algorithms?"
	]

	for query in test_queries:
		print(search_segments(query, limit=3))
		print("\n" + "="*80 + "\n")

# Run the semantic search UI
create_semantic_search_ui()


Testing semantic search with example queries...



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Error searching segments: LanceTable.search() got an unexpected keyword argument 'limit'


Error during search: LanceTable.search() got an unexpected keyword argument 'limit'




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

ERROR:__main__:Error searching segments: LanceTable.search() got an unexpected keyword argument 'limit'


Error during search: LanceTable.search() got an unexpected keyword argument 'limit'




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Error searching segments: LanceTable.search() got an unexpected keyword argument 'limit'


Error during search: LanceTable.search() got an unexpected keyword argument 'limit'




In [16]:
import lancedb
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import logging
import pyarrow as pa
from typing import List, Generator

class SegmentVectorStore:
    def __init__(self, segments_dir: str):
        self.segments_dir = segments_dir
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.db = lancedb.connect("segments.db")
        self.logger = logging.getLogger(__name__)
        self.vector_dim = 384  # MiniLM-L6-v2 embedding dimension
        
    def load_segments(self) -> List[dict]:
        segments = []
        files = os.listdir(self.segments_dir)
        wav_files = [f for f in files if f.endswith('.wav')]
        
        for wav_file in wav_files:
            base_name = wav_file.split('_')[1]
            txt_file = f"segment_{base_name}.txt"
            
            if txt_file in files:
                with open(os.path.join(self.segments_dir, txt_file), 'r', encoding='utf-8') as f:
                    text = f.read().strip()
                
                segments.append({
                    "text": text,
                    "id": f"segment_{base_name}",
                    "audio_file": wav_file
                })
        
        return segments
        
    def create_batches(self, batch_size: int = 20) -> Generator[pa.RecordBatch, None, None]:
        segments = self.load_segments()
        
        # Process segments in batches
        for i in range(0, len(segments), batch_size):
            batch_segments = segments[i:i + batch_size]
            
            # Create embeddings for the batch
            texts = [s["text"] for s in batch_segments]
            embeddings = self.model.encode(texts)
            
            # Prepare arrays for PyArrow
            vectors = pa.array(embeddings.tolist(), pa.list_(pa.float32(), self.vector_dim))
            ids = pa.array([s["id"] for s in batch_segments], pa.string())
            texts = pa.array([s["text"] for s in batch_segments], pa.string())
            audio_files = pa.array([s["audio_file"] for s in batch_segments], pa.string())
            
            # Create and yield batch
            yield pa.RecordBatch.from_arrays(
                [vectors, ids, texts, audio_files],
                ["vector", "id", "text", "audio_file"]
            )
    
    def create_table(self):
        # Define schema
        schema = pa.schema([
            pa.field("vector", pa.list_(pa.float32(), self.vector_dim)),
            pa.field("id", pa.string()),
            pa.field("text", pa.string()),
            pa.field("audio_file", pa.string())
        ])
        
        # Drop existing table if it exists
        if "segments" in self.db.table_names():
            self.db.drop_table("segments")
        
        # Create new table with batches
        table = self.db.create_table(
            "segments",
            self.create_batches(),
            schema=schema,
            mode="create"
        )
        
        return table
    
    def search(self, query: str, limit: int = 5):
        table = self.db.open_table("segments")
        query_vector = self.model.encode(query).tolist()
        
        results = table.search(
            query_vector,
            vector_column_name="vector"
        ).limit(limit).to_list()
        
        return results

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    try:
        store = SegmentVectorStore("segments")
        
        # Create table with batches
        logger.info("Creating table...")
        table = store.create_table()
        logger.info("Table created successfully")
        
        # Test search
        query = "what is this about?"
        logger.info(f"Searching for: {query}")
        results = store.search(query)
        
        print("\nSearch Results:")
        for r in results:
            print(f"\nSegment {r['id']}:")
            print(f"Text: {r['text']}")
            print(f"Audio File: {r['audio_file']}")
            
    except Exception as e:
        logger.error(f"Error: {str(e)}")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Creating table...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:__main__:Table created successfully
INFO:__main__:Searching for: what is this about?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Search Results:

Segment segment_002:
Text: [Music] welcome to computer science 101 in today's video you'll learn the science behind the garbage code you've been writing by learning 101 different computer science terms and concepts this is a computer it's just a piece of tape
Audio File: segment_002_0023519_0037679.wav

Segment segment_009:
Text: applications for that computers have random access memory or ram it's like a neighborhood and inside of every house lives a byte every location has a memory address which the cpu can read and write to you can think of the cpu and ram as
Audio File: segment_009_0108640_0123359.wav

Segment segment_017:
Text: application is to declare a variable this attaches a name to a data point allowing you to reuse it somewhere else in your code python is a dynamically typed language which means we don't need to tell the program exactly which data
Audio File: segment_017_0206878_0220799.wav

Segment segment_048:
Text: what the program does and the outcome 

In [17]:
import lancedb
from sentence_transformers import SentenceTransformer

class SimpleRetriever:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.db = lancedb.connect("segments.db")
        
    def search(self, query: str, limit: int = 5):
        table = self.db.open_table("segments")
        query_vector = self.model.encode(query).tolist()
        
        results = table.search(
            query_vector,
            vector_column_name="vector"
        ).limit(limit).to_list()
        
        return results

def test_retriever():
    # Initialize retriever
    retriever = SimpleRetriever()
    
    # Test queries
    test_queries = [
        "what is this about?",
        "can you explain the main topic?",
        "tell me more about the details"
    ]
    
    # Run searches
    for query in test_queries:
        print(f"\nSearching for: {query}")
        results = retriever.search(query)
        
        print("Results:")
        for r in results:
            print(f"\nSegment {r['id']}:")
            print(f"Text: {r['text']}")
            print(f"Audio File: {r['audio_file']}")

if __name__ == "__main__":
    test_retriever()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2



Searching for: what is this about?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Results:

Segment segment_002:
Text: [Music] welcome to computer science 101 in today's video you'll learn the science behind the garbage code you've been writing by learning 101 different computer science terms and concepts this is a computer it's just a piece of tape
Audio File: segment_002_0023519_0037679.wav

Segment segment_009:
Text: applications for that computers have random access memory or ram it's like a neighborhood and inside of every house lives a byte every location has a memory address which the cpu can read and write to you can think of the cpu and ram as
Audio File: segment_009_0108640_0123359.wav

Segment segment_017:
Text: application is to declare a variable this attaches a name to a data point allowing you to reuse it somewhere else in your code python is a dynamically typed language which means we don't need to tell the program exactly which data
Audio File: segment_017_0206878_0220799.wav

Segment segment_048:
Text: what the program does and the outcome but does

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Results:

Segment segment_002:
Text: [Music] welcome to computer science 101 in today's video you'll learn the science behind the garbage code you've been writing by learning 101 different computer science terms and concepts this is a computer it's just a piece of tape
Audio File: segment_002_0023519_0037679.wav

Segment segment_009:
Text: applications for that computers have random access memory or ram it's like a neighborhood and inside of every house lives a byte every location has a memory address which the cpu can read and write to you can think of the cpu and ram as
Audio File: segment_009_0108640_0123359.wav

Segment segment_052:
Text: inheritance where a subclass can extend and override the behaviors of the parent class and it opens the door to all kinds of other ideas called design patterns now a class by itself doesn't actually do anything instead it's used to
Audio File: segment_052_0635519_0649519.wav

Segment segment_004:
Text: central processing unit if we crack it open w

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Results:

Segment segment_011:
Text: together because we have operating system kernels like linux mac and windows that control all hardware resources via device drivers now to start hacking on the operating system your first entry point is the shell which is a program that exposes the
Audio File: segment_011_0133520_0148400.wav

Segment segment_001:
Text: learn to code and get a high paying job while literally having no idea how anything actually works it all just feels like magic like a pilot driving a giant metal tube in the sky while knowing nothing about aerodynamics
Audio File: segment_001_0011279_0024719.wav

Segment segment_009:
Text: applications for that computers have random access memory or ram it's like a neighborhood and inside of every house lives a byte every location has a memory address which the cpu can read and write to you can think of the cpu and ram as
Audio File: segment_009_0108640_0123359.wav

Segment segment_060:
Text: address is usually alias to a url that is

In [19]:
%pip install sounddevice

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pi

In [22]:
import lancedb
from sentence_transformers import SentenceTransformer
import sounddevice as sd
import soundfile as sf
import os
import time

class SimpleRetriever:
    def __init__(self, audio_dir: str = "segments"):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.db = lancedb.connect("segments.db")
        self.audio_dir = audio_dir
        
    def search(self, query: str, limit: int = 5):
        table = self.db.open_table("segments")
        query_vector = self.model.encode(query).tolist()
        
        results = table.search(
            query_vector,
            vector_column_name="vector"
        ).limit(limit).to_list()
        
        return results
    
    def play_audio(self, audio_filename: str):
        """Play audio file and wait for it to finish"""
        try:
            # Construct full path to audio file
            audio_path = os.path.join(self.audio_dir, audio_filename)
            
            # Load the audio file
            data, samplerate = sf.read(audio_path)
            
            # Play the audio
            print(f"\nPlaying audio: {audio_filename}")
            sd.play(data, samplerate)
            
            # Wait until the audio is finished playing
            status = sd.wait()
            
            if status:
                print("Audio playback completed")
            
        except Exception as e:
            print(f"Error playing audio: {str(e)}")

def test_retriever():
    # Initialize retriever
    retriever = SimpleRetriever()
    
    # Test queries
    test_queries = [
        "what is this about?",
        "can you explain the main topic?",
        "tell me more about the details"
    ]
    
    # Run searches
    for query in test_queries:
        print(f"\nSearching for: {query}")
        results = retriever.search(query)
        
        print("Results:")
        for r in results:
            print(f"\nSegment {r['id']}:")
            print(f"Text: {r['text']}")
            print(f"Audio File: {r['audio_file']}")
            
            # Ask if user wants to play the audio
            play = input(f"Would you like to play this audio segment? (y/n): ")
            if play.lower() == 'y':
                retriever.play_audio(r['audio_file'])
                # Small pause between audio segments
                time.sleep(1)

def interactive_search():
    """Interactive search function that allows users to search and play audio"""
    retriever = SimpleRetriever()
    
    while True:
        # Get search query from user
        query = input("\nEnter your search query (or 'quit' to exit): ")
        
        if query.lower() == 'quit':
            break
            
        # Perform search
        results = retriever.search(query)
        
        # Display results
        for i, r in enumerate(results, 1):
            print(f"\n{i}. Segment {r['id']}:")
            print(f"Text: {r['text']}")
            print(f"Audio File: {r['audio_file']}")
        
        # Ask which result to play
        while True:
            choice = input("\nEnter the number of the segment to play (or 'n' for next search): ")
            
            if choice.lower() == 'n':
                break
                
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(results):
                    retriever.play_audio(results[choice_idx]['audio_file'])
                else:
                    print("Invalid selection. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'n'.")

if __name__ == "__main__":
    print("1. Run test queries")
    print("2. Interactive search")
    choice = input("Select mode (1 or 2): ")
    
    if choice == "1":
        test_retriever()
    elif choice == "2":
        interactive_search()
    else:
        print("Invalid choice")

1. Run test queries
2. Interactive search


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2



Searching for: what is this about?


C:\Users\shawn\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Results:

Segment segment_002:
Text: [Music] welcome to computer science 101 in today's video you'll learn the science behind the garbage code you've been writing by learning 101 different computer science terms and concepts this is a computer it's just a piece of tape
Audio File: segment_002_0023519_0037679.wav

Playing audio: segment_002_0023519_0037679.wav

Segment segment_009:
Text: applications for that computers have random access memory or ram it's like a neighborhood and inside of every house lives a byte every location has a memory address which the cpu can read and write to you can think of the cpu and ram as
Audio File: segment_009_0108640_0123359.wav

Segment segment_017:
Text: application is to declare a variable this attaches a name to a data point allowing you to reuse it somewhere else in your code python is a dynamically typed language which means we don't need to tell the program exactly which data
Audio File: segment_017_0206878_0220799.wav

Segment segment_048:
Text

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Results:

Segment segment_002:
Text: [Music] welcome to computer science 101 in today's video you'll learn the science behind the garbage code you've been writing by learning 101 different computer science terms and concepts this is a computer it's just a piece of tape
Audio File: segment_002_0023519_0037679.wav

Segment segment_009:
Text: applications for that computers have random access memory or ram it's like a neighborhood and inside of every house lives a byte every location has a memory address which the cpu can read and write to you can think of the cpu and ram as
Audio File: segment_009_0108640_0123359.wav

Segment segment_052:
Text: inheritance where a subclass can extend and override the behaviors of the parent class and it opens the door to all kinds of other ideas called design patterns now a class by itself doesn't actually do anything instead it's used to
Audio File: segment_052_0635519_0649519.wav

Segment segment_004:
Text: central processing unit if we crack it open w

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Results:

Segment segment_011:
Text: together because we have operating system kernels like linux mac and windows that control all hardware resources via device drivers now to start hacking on the operating system your first entry point is the shell which is a program that exposes the
Audio File: segment_011_0133520_0148400.wav

Segment segment_001:
Text: learn to code and get a high paying job while literally having no idea how anything actually works it all just feels like magic like a pilot driving a giant metal tube in the sky while knowing nothing about aerodynamics
Audio File: segment_001_0011279_0024719.wav

Segment segment_009:
Text: applications for that computers have random access memory or ram it's like a neighborhood and inside of every house lives a byte every location has a memory address which the cpu can read and write to you can think of the cpu and ram as
Audio File: segment_009_0108640_0123359.wav

Segment segment_060:
Text: address is usually alias to a url that is

In [29]:
import lancedb
from sentence_transformers import SentenceTransformer
import sounddevice as sd
import soundfile as sf
import os
import time

class SimpleRetriever:
    def __init__(self, audio_dir: str = "segments"):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.db = lancedb.connect("segments.db")
        self.audio_dir = audio_dir
        
    def search(self, query: str, limit: int = 5):
        table = self.db.open_table("segments")
        query_vector = self.model.encode(query).tolist()
        
        results = table.search(
            query_vector,
            vector_column_name="vector"
        ).limit(limit).to_list()
        
        return results
    
    def play_audio(self, audio_filename: str):
        """Play audio file and wait for it to finish"""
        try:
            # Construct full path to audio file
            audio_path = os.path.join(self.audio_dir, audio_filename)
            
            # Load the audio file
            data, samplerate = sf.read(audio_path)
            
            # Play the audio
            print(f"\nPlaying audio: {audio_filename}")
            sd.play(data, samplerate)
            
            # Wait until the audio is finished playing
            status = sd.wait()
            
            if status:
                print("Audio playback completed")
            
        except Exception as e:
            print(f"Error playing audio: {str(e)}")

def test_retriever():
    # Initialize retriever
    retriever = SimpleRetriever()
    
    # Test queries
    test_queries = [
        "what is this about?",
        "can you explain the main topic?",
        "tell me more about the details"
    ]
    
    # Run searches
    for query in test_queries:
        print(f"\nSearching for: {query}")
        results = retriever.search(query)
        
        print("Results:")
        for r in results:
            print(f"\nSegment {r['id']}:")
            print(f"Text: {r['text']}")
            print(f"Audio File: {r['audio_file']}")
            
            # Ask if user wants to play the audio
            play = input(f"Would you like to play this audio segment? (y/n): ")
            if play.lower() == 'y':
                retriever.play_audio(r['audio_file'])
                # Small pause between audio segments
                time.sleep(1)

def interactive_search():
    """Interactive search function that allows users to search and play audio"""
    retriever = SimpleRetriever()
    
    while True:
        # Get search query from user
        query = input("\nEnter your search query (or 'quit' to exit): ")
        
        if query.lower() == 'quit':
            break
            
        # Perform search
        results = retriever.search(query)
        
        # Display results
        for i, r in enumerate(results, 1):
            print(f"\n{i}. Segment {r['id']}:")
            print(f"Text: {r['text']}")
            print(f"Audio File: {r['audio_file']}")
        
        # Ask which result to play
        while True:
            choice = input("\nEnter the number of the segment to play (or 'n' for next search): ")
            
            if choice.lower() == 'n':
                break
                
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(results):
                    retriever.play_audio(results[choice_idx]['audio_file'])
                else:
                    print("Invalid selection. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'n'.")

if __name__ == "__main__":
    print("1. Run test queries")
    print("2. Interactive search")
    choice = input("Select mode (1 or 2): ")
    
    if choice == "1":
        test_retriever()
    elif choice == "2":
        interactive_search()
    else:
        print("Invalid choice")

1. Run test queries
2. Interactive search


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


1. Segment segment_000:
Text: what's the first thing you should do when your code throws an error obviously you should change nothing and try to run it again a few times if that doesn't work you're gonna need a computer science degree the awesome thing about software engineering is that you can
Audio File: segment_000_0000080_0013359.wav

2. Segment segment_041:
Text: memory for executing your code when a function keeps calling itself the language will keep pushing frames onto the call stack until you get a stack overflow error to avoid this your algorithm needs a base condition so it knows when to terminate the loop now
Audio File: segment_041_0499839_0513438.wav

3. Segment segment_038:
Text: while loop will run this block of code over and over again until the condition in the parentheses becomes false that can be useful but more often than not you'll want to loop over an iterable data type like an array most languages have a for loop that can run some code
Audio File: segment_038_0